Herzlich Willkommen, liebe Datendetektivinnen.
Um die Daten besser zu verstehen, laden wir zuerst unseren Werzeugkoffer:

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
print("Alle Werzeuge erfolgreich geladen")

Jetzt laden wir die eigentlichen Daten aus einer Tabelle. Dies sind die Daten mit fehlenden/falschen Angaben.

In [ ]:
file = "GirlsDay_Beispiel_Windanlagen_v4_missing.csv"
daten = pd.read_csv(file)
#Wir zaehlen, wie viele Daten wir je Katgorie haben.
daten.count()

Jetzt wollen wir uns aber einmal ein paar Zeieln dieser Tabelle probehalber angucken. Wie können wir das machen?

Zum Glück gibt es da eine weitere hilfreiche Funktion in Pandas: `.head(n)`

Hiermit kannst du dir die n ersten Zeilen deiner Tabelle ausgeben lassen, also den Kopf. Logisch, wie englisch "head" auch schon sagt. Wollen wir also die ersten 10 Zeilen sehen, schreiben wir `.head(n=10)`. Wollen wir die ersten 50 Zeilen haben, dann `.head(n=50)`. Das gleich geht auch wenn du dir die letzten Zeilen von unten angucken willst. Dafür kannst du `.tail(n)` benutzen. Willst du also die 10 untersten Zeilen sehen, was bentzt du dann? Probier es aus!

In [ ]:

daten.head(n=10)
#

NaN steht für "Not a Number", also "keine Zahl". So sagt der Computer, dass er diese Eingabe nicht versteht.

Nun lass uns ein wenig eine Übersicht über die Daten bekommen. Hierfür hat Pandas eine Funktion die sich `.describe()` nennt. Das gibt dir dann eine zusammenfassende Beschreibung deiner einzelnen Spalten aus. Das sind dann ganz viele mathematische Zusammenfassende Werte... Vielleicht kennt ihr ja den ein oder anderen?

Was bedeuten denn zum Beispiel min & max?

In [ ]:
daten.describe()

Die Funktion `.sum()` summiert alle Einträge. So können wir z.B. die Gesamtleistung bestimmen.

In [ ]:
daten['Leistung'].sum()

Wir können Daten auch visuell darstellen. Dies hilft uns bei dem Verständnis und beim weiteren ausfüllen.

In [ ]:
plt.scatter(daten["ID"].values,daten["Hoehe"].values)
plt.xlabel("ID")
plt.ylabel("Hoehe [m]")
plt.show()

Wir sehen dass die meisten Daten eng beisammen liegen und es 6 Ausreisser gibt.
Die Daten die eng beisammen liegen scheinen aufsteigend: größere ID bedeutet höhere Nabenhöhe.
Wir vermuten, dass die Daten der Nabenhöhe nach sortiert sind.
Hilft uns diese Information um die fehlenden Nabenhöhen zu ergänzen?


Funktion die LeistungProFläche berechnet basierend auf Leistung und Durchmesser.

In [ ]:
def LeistungProFlaeche(Leistung_MW, Durchmesser):
    Flaeche = 3.14*(Durchmesser/2)**2
    Leistung_Watt = Leistung_MW*1000000
    return np.round(Leistung_Watt/Flaeche)

Beispiel erste Zeile der Datentabelle

In [ ]:
print(LeistungProFlaeche(Leistung_MW=4, Durchmesser=137))

Ein Beispiel mit der ID=7

In [ ]:
print(LeistungProFlaeche(Leistung_MW=4, Durchmesser=145))

Du kannst die Werte ändern, um dir die fehlenden Daten zu berechnen

Funktion um den Durchmesser zu berechnen, basierend auf Leistung und LeistungProFlaeche

In [ ]:
def Durchmesser(Leistung_MW, Leistung_pro_flaeche):
    flaeche = 1000000*Leistung_MW/Leistung_pro_flaeche
    return np.round(2*np.sqrt(flaeche/np.pi))

Dafür ein Beispiel für die zweite Zeile.

In [ ]:
print(Durchmesser(Leistung_MW=1, Leistung_pro_flaeche=90))

Und ein Beispiel für ID=3:

In [ ]:
print(Durchmesser(Leistung_MW=2, Leistung_pro_flaeche=167))
#

Du kannst die Werte ändern, um dir die fehlenden Daten zu berechnen.

### GRUPPIEREN DER DATEN

Wir laden nun die korrigierten Daten ein.

In [ ]:
file = "GirlsDay_Beispiel_Windanlagen_v4_complete.csv"
daten = pd.read_csv(file)

Wir stellen die Daten visuell dar.

In [ ]:
plt.scatter(daten["Hoehe"].values,daten["LeistungProFlaeche"].values)
plt.xlabel("Hoehe [m]")
plt.ylabel("Leistung pro Flaeche [W/m$^2$]")
plt.show()

Frage: Wie würdet ihr clustern? Wie viele Gruppen würdet ihr empfehlen?

Dann lasst uns doch mal Künstliche Intelligenz nutzen um die Daten zu clustern! Bereit?

In [ ]:
std1 = daten["Hoehe"].std()
std2 = daten["LeistungProFlaeche"].std()
print(std1,std2)
# Hier ein paar nötige Vorbereitungen der Daten...
daten["Hoehe"] /= std1
daten["LeistungProFlaeche"] /= std2

#### K-means Clustering

Was ist die optimale Anzahl an Gruppen?
Wir probieren verschiedene Anzahlen aus:

In [ ]:

for Nclusters in [2,3,4,5]:
    print("Anzahl Gruppen:",Nclusters)
    kmeans = KMeans(n_clusters=Nclusters).fit(daten[['Hoehe','LeistungProFlaeche']])
    centroids = kmeans.cluster_centers_
#    print(centroids)

    plt.scatter(daten["Hoehe"]*std1, daten["LeistungProFlaeche"]*std2, c=kmeans.labels_.astype(float), s=100, alpha=1.0)
#    plt.scatter(centroids[:, 0]*mean2, centroids[:, 1]*mean1, c='red', s=50)
    plt.xlabel("Hoehe [m]")
    plt.ylabel("Leistung pro Flaeche [W/m$^2$]")
    plt.show()

Robert Habeck will kleine Windenergieanlagen (geringe Höhe) mit hoher Leistung pro Fläche. Welche Gruppe wuerdet ihr ihm dazu empfehlen? Wenn ihr die Gruppe identifiziert habt, könnt ihr in der Tabelle schauen, ob diese Anlagen etwas gemeinsam haben.